<a href="https://colab.research.google.com/github/SahilKureshi/Training/blob/main/pytorch_and_finetunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
# from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [ ]:
df = pd.read_csv('/content/fashion-mnist_train.csv')

In [ ]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0,0,0,0,0,0,0,5,0,...,0.0,0.0,0.0,30.0,43.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,1,2,0,0,0,0,...,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X = df.drop('label', axis=1)
y = df['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train/255
X_test = X_test/255

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]


In [ ]:
train_dataset = CustomDataset(X_train.values, y_train.values)
test_dataset = CustomDataset(X_test.values, y_test.values)

In [ ]:
len(train_dataset)

2648

In [ ]:
class MyNN(nn.Module):
  def __init__(self, input_dim, ouput_dim, num_hidden_layers, neurons_per_layer, dropout_rate):
    super().__init__()
    layers = []
    for i in range(num_hidden_layers):
      layers.append(nn.Linear(input_dim, neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout_rate))
      input_dim = neurons_per_layer
    layers.append(nn.Linear(input_dim, ouput_dim))
    self.layers = nn.Sequential(*layers)

  def forward(self, x):
    return self.layers(x)


In [ ]:
# objective function create an objective function specify your all objective and range of varibale to run on
def objective(trial):

  num_hidden_layers = trial.suggest_int('num_hidden_layers', 1,5)
  neurons_per_layer = trial.suggest_int('neurons_per_layer', 16, 128, step=8)
  epochs = trial.suggest_int('epochs', 10, 50, step=10)
  dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5, step=0.1)
  learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1)
  batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
  optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])
  weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-1)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

  model = MyNN(input_dim=784, ouput_dim=10, num_hidden_layers=num_hidden_layers, neurons_per_layer=neurons_per_layer, dropout_rate=dropout_rate)
  model.to(device)

  criterion = nn.CrossEntropyLoss()
  if optimizer_name == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimizer_name == 'SGD':
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

  for epoch in range(epochs):
    for batch_features, labels in train_loader:
      batch_features = batch_features.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      outputs = model(batch_features)
      loss = criterion(outputs, labels)

      loss.backward()
      optimizer.step()


  model.eval()
  total = 0
  correct = 0

  with torch.no_grad():
    for batch_features, batch_labels in test_loader:
      batch_features = batch_features.to(device)
      batch_labels = batch_labels.to(device)
      outputs = model(batch_features)
      _, predicted = torch.max(outputs,1)

      total = total+batch_features.shape[0]

      correct = correct + (predicted == batch_labels).sum().item()

  accuracy = correct/total
  return accuracy



In [ ]:
!pip install optuna

In [ ]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2025-07-30 09:35:51,523] A new study created in memory with name: no-name-c1d013e4-7b29-4500-969c-17628bc157e5
[I 2025-07-30 09:36:00,453] Trial 0 finished with value: 0.08597285067873303 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 96, 'epochs': 40, 'dropout_rate': 0.5, 'learning_rate': 0.07210906029077757, 'batch_size': 128, 'optimizer': 'SGD', 'weight_decay': 0.020885961453048666}. Best is trial 0 with value: 0.08597285067873303.
[I 2025-07-30 09:36:04,281] Trial 1 finished with value: 0.08597285067873303 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 32, 'epochs': 50, 'dropout_rate': 0.2, 'learning_rate': 0.09197920365105192, 'batch_size': 128, 'optimizer': 'Adam', 'weight_decay': 0.04445829848109961}. Best is trial 0 with value: 0.08597285067873303.
[I 2025-07-30 09:36:05,920] Trial 2 finished with value: 0.08597285067873303 and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 112, 'epochs': 20, 'dropout_rate': 0.5, 'learning_rate': 0.00

In [ ]:
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

Best hyperparameters: {'num_hidden_layers': 5, 'neurons_per_layer': 96, 'epochs': 40, 'dropout_rate': 0.5, 'learning_rate': 0.07210906029077757, 'batch_size': 128, 'optimizer': 'SGD', 'weight_decay': 0.020885961453048666}
